In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, AveragePooling2D, Activation, ELU, BatchNormalization, Layer, Lambda, LeakyReLU
from keras.optimizers import Adadelta, Adam, RMSprop
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets.mnist import load_data as load
from keras.models import load_model
import pandas as pd
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras import backend as K
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [100]:
train_data = np.loadtxt("train.csv", delimiter=',', skiprows=1)
test_data = np.loadtxt("test.csv", delimiter=',', skiprows=1)

In [101]:
train_input = train_data[:, 1:].astype('float32')/255
train_target = train_data[:, 0].astype('int32')
print("Size of train dataset: {}".format(train_input.shape))

test_input = test_data.astype('float32')/255
print("Size of test dataset: {}".format(test_input.shape))

Size of train dataset: (42000, 784)
Size of test dataset: (28000, 784)


In [102]:
train_input = train_input.reshape(train_input.shape[0], 28, 28, 1)
test_input = test_input.reshape(test_input.shape[0], 28, 28, 1)

In [103]:
train_target = to_categorical(train_target)

In [74]:
def build_model():
    model = Sequential()

    model.add(Conv2D(64, kernel_size=(3, 3), padding='same', input_shape=(28,28,1)))
    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5, gamma_initializer="uniform"))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Conv2D(64, kernel_size=(3, 3), padding='same'))
    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5, gamma_initializer="uniform"))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))
    
    model.add(Conv2D(128, kernel_size=(3, 3), padding='same'))
    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5, gamma_initializer="uniform"))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Conv2D(128, kernel_size=(3, 3), padding='same'))
    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5, gamma_initializer="uniform"))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))
    
    model.add(Conv2D(256, kernel_size=(3, 3), padding='same'))
    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5, gamma_initializer="uniform"))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Conv2D(256, kernel_size=(3, 3), padding='same'))
    model.add(BatchNormalization(momentum=0.9, epsilon=1e-5, gamma_initializer="uniform"))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(10, activation='softmax'))
    
    return model

In [75]:
optimizer = RMSprop(lr=0.001, rho=0.92, epsilon=1e-08, decay=0.0)

In [ ]:
adm = Adam(learning_rate=1e-3, beta_1=0.92, beta_2=0.999, amsgrad=False)
model = build_model()

model.compile(loss = 'categorical_crossentropy', optimizer = adm, metrics=['accuracy'])

In [65]:
x_train, x_validation, y_train, y_validation = train_test_split(train_input, train_target, test_size=0.2 , random_state=42)

In [95]:
data_gen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False, 
        samplewise_std_normalization=False,
        zca_whitening=False,  
        rotation_range=10,  
        zoom_range = 0.1,  
        width_shift_range=0.1,
        height_shift_range=0.1, 
        horizontal_flip=False, 
        vertical_flip=False)  

data_gen.fit(x_train)

In [96]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.25, 
                                            min_lr=0.00001)

es = EarlyStopping(monitor='val_loss', min_delta=1e-1, verbose=1, patience=100, restore_best_weights=True)

In [97]:
batch_size = 128
epochs = 50

In [98]:
history = model.fit_generator(data_gen.flow(x_train, y_train, batch_size=batch_size), 
                              validation_data=data_gen.flow(x_validation, y_validation, batch_size=batch_size), 
                              steps_per_epoch=len(x_train)//batch_size, epochs=epochs, callbacks=[learning_rate_reduction, es])

Epoch 1/50
468/468 [==============================] - 69s 147ms/step - loss: 0.2388 - accuracy: 0.9235 - val_loss: 0.0410 - val_accuracy: 0.9795
Epoch 2/50
468/468 [==============================] - 64s 137ms/step - loss: 0.0811 - accuracy: 0.9743 - val_loss: 0.0042 - val_accuracy: 0.9880
Epoch 3/50
468/468 [==============================] - 64s 137ms/step - loss: 0.0637 - accuracy: 0.9800 - val_loss: 0.3948 - val_accuracy: 0.9864
Epoch 4/50
468/468 [==============================] - 64s 138ms/step - loss: 0.0572 - accuracy: 0.9818 - val_loss: 2.5364e-04 - val_accuracy: 0.9886
Epoch 5/50
468/468 [==============================] - 64s 137ms/step - loss: 0.0495 - accuracy: 0.9848 - val_loss: 0.0047 - val_accuracy: 0.9899
Epoch 6/50
468/468 [==============================] - 64s 137ms/step - loss: 0.0475 - accuracy: 0.9854 - val_loss: 0.0012 - val_accuracy: 0.9918
Epoch 7/50
468/468 [==============================] - 64s 137ms/step - loss: 0.0443 - accuracy: 0.9865 - val_loss: 0.1412 - va

In [99]:
model.save("my_model")

In [105]:
results = model.predict(test_input)
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)
submission.to_csv("my_submission.csv",index=False)